In [1]:
%load_ext autoreload
%autoreload 2
import netCDF4
import xarray as xr
import numpy as np
from pathlib import Path
from hmpai.pytorch.models import *
from hmpai.training import split_data_on_participants
from hmpai.pytorch.training import train, validate, calculate_class_weights, train_and_test, k_fold_cross_validate, test
from hmpai.pytorch.generators import SAT1Dataset
from hmpai.data import SAT1_STAGES_ACCURACY, SAT_CLASSES_ACCURACY
from hmpai.visualization import plot_confusion_matrix
from hmpai.normalization import *
from torchinfo import summary
from torchvision.transforms import Compose
from hmpai.pytorch.transforms import *
from hmpai.utilities import print_results, CHANNELS_2D, AR_SAT1_CHANNELS
from torch.utils.data import DataLoader
from mne.io import read_info
from pathlib import Path
from hmpai.pytorch.models import *
from hmpai.training import split_data_on_participants
import pandas as pd
from hmpai.pytorch.utilities import DEVICE, set_global_seed, get_summary_str, save_model, load_model

import os
DATA_PATH = Path(os.getenv("DATA_PATH"))

In [2]:
set_global_seed(42)
data_path_sat2 = DATA_PATH / "sat2/split_stage_data_100hz.nc"
dataset_sat2 = xr.open_dataset(data_path_sat2)
# data_path_sat1 = DATA_PATH / "sat1/stage_data_100hz.nc"
# dataset_sat1 = xr.load_dataset(data_path_sat1)
# dataset_sat1 = dataset_sat1.rename_vars({'RT': 'rt'})

# dataset_sat2 = dataset_sat2.where(dataset_sat2.event_name.str.contains("accuracy"), drop=True)
# dataset_sp = dataset_sat1.where(dataset_sat1.event_name.str.contains("speed"), drop=True)

In [3]:
# shape_topological = False
# info_to_keep = ['rt']
# train_data_sat2, val_data_sat2, test_data_sat2 = split_data_on_participants(
#     dataset_sat2, 60, norm_min1_to_1
# )
# train_data_sat1, val_data_sat1, test_data_sat1 = split_data_on_participants(
#     dataset_sat1, 60, norm_min1_to_1
# )
# train_data = xr.merge([train_data_sat2, train_data_sat1])
# val_data = xr.merge([val_data_sat2, val_data_sat1])
# test_data = xr.merge([test_data_sat2, test_data_sat1])
# train_dataset = SAT1Dataset(train_data, shape_topological=shape_topological, labels=SAT_CLASSES_ACCURACY, set_to_zero=True, info_to_keep=info_to_keep, order_by_rt=False)
# val_dataset = SAT1Dataset(val_data, shape_topological=shape_topological, labels=SAT_CLASSES_ACCURACY, set_to_zero=True, info_to_keep=info_to_keep, order_by_rt=False)
# test_dataset = SAT1Dataset(test_data, shape_topological=shape_topological, labels=SAT_CLASSES_ACCURACY, set_to_zero=True, info_to_keep=info_to_keep, order_by_rt=False)

In [13]:
shape_topological = False
info_to_keep = ["rt"]
train_data_sat2, val_data_sat2, test_data_sat2 = split_data_on_participants(
    dataset_sat2, 60, norm_min1_to_1
)
# train_data_sat2 = train_data_sat2.where(train_data_sat2.labels != 0, drop=True)
# val_data_sat2 = val_data_sat2.where(val_data_sat2.labels != 0, drop=True)
# test_data_sat2 = test_data_sat2.where(test_data_sat2.labels != 0, drop=True)
# train_data_sat1, val_data_sat1, test_data_sat1 = split_data_on_participants(
#     dataset_sat1, 60, norm_min1_to_1
# )
# train_data = xr.merge([train_data_sat2, train_data_sat1])
# val_data = xr.merge([val_data_sat2, val_data_sat1])
# test_data = xr.merge([test_data_sat2, test_data_sat1])
train_dataset = SAT1Dataset(
    train_data_sat2,
    shape_topological=shape_topological,
    labels=SAT_CLASSES_ACCURACY,
    set_to_zero=True,
    info_to_keep=info_to_keep,
    order_by_rt=False,
    transform=Compose([RandomCropTransform(), ChannelsDropout(0.5, 0.4)]),
)
val_dataset = SAT1Dataset(
    val_data_sat2,
    shape_topological=shape_topological,
    labels=SAT_CLASSES_ACCURACY,
    set_to_zero=True,
    info_to_keep=info_to_keep,
    order_by_rt=False,
    # transform=Compose([RandomCropTransform()]),
)
test_dataset = SAT1Dataset(
    test_data_sat2,
    shape_topological=shape_topological,
    labels=SAT_CLASSES_ACCURACY,
    set_to_zero=True,
    info_to_keep=info_to_keep,
    order_by_rt=False,
    # transform=Compose([RandomCropTransform()]),
)

['S11', 'S18', 'S4', 'S9', 'S5', 'S1', 'S15', 'S13']


In [15]:
# Train model
model = MambaModel(embed_dim=256, n_channels=30, n_classes=5, n_layers=6, global_pool=False)
train_and_test(
    model,
    train_dataset,
    test_dataset,
    val_dataset,
    logs_path=Path("../logs/"),
    workers=0,
    batch_size=64,
    labels=SAT_CLASSES_ACCURACY,
    epochs=100,
    # epochs=1,
    weight_decay=0.001,
    label_smoothing=0.1,
    lr=0.00001
)

  0%|          | 0/154 [00:00<?, ? batch/s]

              precision    recall  f1-score   support

           0       1.00      0.97      0.98    401682
           1       0.51      0.25      0.33     39184
           2       0.52      0.09      0.15    107148
           3       0.32      0.08      0.12    106627
           4       0.07      0.89      0.13     17307

    accuracy                           0.64    671948
   macro avg       0.48      0.45      0.34    671948
weighted avg       0.76      0.64      0.65    671948



  0%|          | 0/154 [00:00<?, ? batch/s]

              precision    recall  f1-score   support

           0       1.00      0.97      0.99    409912
           1       0.46      0.36      0.40     39184
           2       0.49      0.41      0.45    107148
           3       0.43      0.36      0.39    106627
           4       0.09      0.40      0.15     17307

    accuracy                           0.74    680178
   macro avg       0.50      0.50      0.48    680178
weighted avg       0.78      0.74      0.75    680178



  0%|          | 0/154 [00:00<?, ? batch/s]

...

              precision    recall  f1-score   support

           0       1.00      0.98      0.99    408453
           1       0.68      0.12      0.20     39184
           2       0.48      0.33      0.39    107148
           3       0.45      0.06      0.11    106627
           4       0.08      0.81      0.14     17307

    accuracy                           0.68    678719
   macro avg       0.54      0.46      0.37    678719
weighted avg       0.79      0.68      0.69    678719



  0%|          | 0/154 [00:00<?, ? batch/s]

              precision    recall  f1-score   support

           0       1.00      0.98      0.99    407641
           1       0.71      0.25      0.37     39184
           2       0.55      0.20      0.29    107148
           3       0.42      0.05      0.09    106627
           4       0.07      0.89      0.13     17307

    accuracy                           0.66    677907
   macro avg       0.55      0.47      0.37    677907
weighted avg       0.80      0.66      0.68    677907



  0%|          | 0/154 [00:00<?, ? batch/s]

KeyboardInterrupt: 

In [6]:
model = Seq2SeqTransformer(d_model=30, ff_dim=1024, num_heads=8, num_layers=6, num_classes=len(SAT_CLASSES_ACCURACY), emb_dim=512)
model.set_pretraining(False)
train_and_test(
    model,
    train_dataset,
    test_dataset,
    val_dataset,
    logs_path=Path("../logs/"),
    workers=4,
    batch_size=64,
    labels=SAT_CLASSES_ACCURACY,
    epochs=10,
    # epochs=1,
    weight_decay=0.001,
    label_smoothing=0.1,
    lr=0.00001
)

  0%|          | 0/167 [00:00<?, ? batch/s]

ERROR: Unexpected bus error encountered in worker. This might be caused by insufficient shared memory (shm).
 ERROR: Unexpected bus error encountered in worker. This might be caused by insufficient shared memory (shm).
 ERROR: Unexpected bus error encountered in worker. This might be caused by insufficient shared memory (shm).
 ERROR: Unexpected bus error encountered in worker. This might be caused by insufficient shared memory (shm).
 ERROR: Unexpected bus error encountered in worker. This might be caused by insufficient shared memory (shm).
 ERROR: Unexpected bus error encountered in worker. This might be caused by insufficient shared memory (shm).
 ERROR: Unexpected bus error encountered in worker. This might be caused by insufficient shared memory (shm).
 ERROR: Unexpected bus error encountered in worker. This might be caused by insufficient shared memory (shm).
 

RuntimeError: DataLoader worker (pid(s) 166091, 166096, 166097, 166098, 166099) exited unexpectedly